In [1]:
import sys
import os
import numpy as np
import random
import json
import string
import itertools
import math
from Crypto.Util.number import long_to_bytes as ltb, bytes_to_long as btl, inverse
from pwn import remote, process, log, args, debug, info, log as pwnLog
from tqdm import tqdm, trange
from matplotlib import pyplot as plt
from functools import reduce, cache
from base64 import b64encode, b64decode
from collections import Counter

In [2]:
# Uncomment below line to use sagemath module
from sage.all_cmdline import *


In [3]:
n = 137695652953436635868173236797773337408441001182675256086214756367750388214098882698624844625677992374523583895607386174643756159168603070583418054134776836804709359451133350283742854338177917816199855370966725059377660312824879861277400624102267119229693994595857701696025366109135127015217981691938713787569
leak = 6745414226866166172286907691060333580739794735754141517928503510445368134531623057
ct = 60939585660386801273264345336943282595466297131309357817378708003135300231065734017829038358019271553508356563122851120615655640023951268162873980957560729424913748657116293860815453225453706274388027182906741605930908510329721874004000783548599414462355143868922204060850666210978837231187722295496753756990


Reference: 
- https://eprint.iacr.org/2020/1506.pdf
- https://github.com/jvdsn/crypto-attacks/blob/master/attacks/factorization/coppersmith.py

In [4]:
sys.path.insert(1, '/mnt/d/code/Helper') # git clone it first and change your path
from crypto_attacks.factorization.coppersmith import factorize_p
from shared.partial_integer import PartialInteger
factorize_p(
    n, # modulus 
    PartialInteger.parse_be(f"{leak:0272b}" + '?'*240, 2), # f(x) = (leak * 2^240) + x
    0.5, # beta for solving equation on f(x) mod n^beta
    0.03  # decrease this for higher shift => Bigger lattice dimension => Take more computation power but more accuracy
)


(08-May-23 15:01:11) root-INFO: Trying m = 9, t = 9...
(08-May-23 15:01:11) root-DEBUG: Generating shifts...
(08-May-23 15:01:11) root-DEBUG: Creating a lattice with 18 shifts (order = None, sort_shifts_reverse = False, sort_monomials_reverse = False)...
(08-May-23 15:01:11) root-DEBUG: Reducing a 18 x 18 lattice...
(08-May-23 15:01:11) root-DEBUG: Reconstructing polynomials (divide_original = True, modulus_bound = True, divide_gcd = True)...
(08-May-23 15:01:11) root-DEBUG: Original polynomial divides reconstructed polynomial at row 0, dividing...
(08-May-23 15:01:11) root-DEBUG: Original polynomial divides reconstructed polynomial at row 1, dividing...
(08-May-23 15:01:11) root-DEBUG: Reconstructed 18 polynomials
(08-May-23 15:01:11) root-DEBUG: Using univariate polynomial to find roots...


(11918115327452840353182098808413773161526179795462517390630981598332371960071647247964792629975772037391372830863318764111397518777467149584983329384153671,
 11553475459014976853548967468638172498419788021623802974508623943234306723640407777491109179572222419366963601151005006800409206893050401617294252857306439)

## Decrypting flag

In [5]:
p = 11918115327452840353182098808413773161526179795462517390630981598332371960071647247964792629975772037391372830863318764111397518777467149584983329384153671
q = 11553475459014976853548967468638172498419788021623802974508623943234306723640407777491109179572222419366963601151005006800409206893050401617294252857306439

ltb(pow(ct, inverse(0x10001, (p-1)*(q-1)), n))

b'HCMUS-CTF{d0nt_b3_4n_3XhiB1ti0ni5t_0r_y0uLL_g3t_eXp0s3d}'